In [5]:
import pathlib
from gemd.entity import ProcessTemplate, ParameterTemplate, RealBounds, CategoricalBounds, ConditionTemplate
from gemd.entity import ProcessSpec, PerformedSource, Condition, NominalReal, NominalCategorical, Parameter
from gemd.entity.util import make_instance
from gemd.json import GEMDJson

## For XRD copper

In [98]:
#One cell to define a dictionary of all the parameters

creating_puck = {
    'name': 'Julia',
    'date': '7/15/2022',
    'glass-slide thickness': '2.0', # in units of mm
    'vaseline name': 'Vaspure',
    'amount of vaseline': '6', # in units of l
    'humidity': '0.08', # fractional, from 0 to 1
}

In [99]:
humidity_templ = ConditionTemplate(
    'humidity',
    bounds=RealBounds(0.,1.0,'dimensionless')
)

temp = ProcessTemplate(
    'creating puck',
    parameters=[
        ParameterTemplate(
            'glass thickness',
            bounds=RealBounds(0.0,10.0,'mm')
        ),
        ParameterTemplate(
            'amount of vaseline',
            bounds=RealBounds(0.0,15.0,'ml')
        ),
        ParameterTemplate(
            'name of vaseline',
            bounds=CategoricalBounds(['Vaspure','Vaseline','Giant','Nivea'])
        )
    ],
)

In [101]:
spec = ProcessSpec(
    name='creating puck',
    template=temp,
    conditions=[],
    parameters=[
        Parameter(
            'glass thickness',
            value=NominalReal(float(creating_puck['glass-slide thickness']),'mm'),
        ),
        Parameter(
            'amount of vaseline',
            value=NominalReal(float(creating_puck['amount of vaseline']),'ml')
        ),
        Parameter(
            'name of vaseline',
            value=NominalCategorical(creating_puck['vaseline name'])
        )
    ],
)

In [103]:
run = make_instance(spec)
run.source = PerformedSource(creating_puck['name'],creating_puck['date'])
run.conditions.append(
    Condition(
            'humidity',
            value=NominalReal(float(creating_puck['humidity']),'kg/kg'),
            template=humidity_templ
        )
)

In [110]:
encoder = GEMDJson()

fn = 'example_process_template.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(temp,indent=2))

fn = 'example_process_spec.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(spec,indent=2))

fn = 'example_process_run.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(run,indent=2))

entire_model_fn = 'entire_model.json'
with open(pathlib.Path()/entire_model_fn,'w') as fp :
    fp.write(encoder.dumps([run],indent=2))


In [115]:
with open(pathlib.Path()/entire_model_fn,'r') as fp :
    runs = encoder.load(fp)
run = runs[0]

In [117]:
run.spec==spec

True

## Electric Transport PPMS

In [6]:
PPMS_ET_setup = {
    'name': 'Julia',
    'date': '17/07/2022',
    #conditions
    'voltage orientation': 'parallel', #either label 'parallel' to curent or if perpendicualr (hall resistivity) 
                            #then state either '+V on top' or '+V on bottom'
    'humidity level in lab': '0.08',
    #parameters 
    'current': '6',
    'sample dimentions': '50:60', # number x: number y in mm
    'compound stochiometry': 'NO2' } #in capital letteres 

#should I add an extra category asking about whether they are looking for resistivity or hall resistivity?

In [9]:
#set up templates
temp_ET = ProcessTemplate(
    'Eletric Transport PPMS setup',
    parameters=[
        
        ParameterTemplate('humidity level in lab', bounds=RealBounds(0.0,1.0,'kg/kg') ),
        
        ParameterTemplate('voltage orientation', bounds=CategoricalBounds(['+V on top', '+V on bottom','parallel',]) )
    ],
)
#is it possible to set bounderies on capital letters and numbers?

In [17]:
#set up spec
spec_ET = ProcessSpec(
    name='Eletric Transport PPMS setup',
    template=temp_ET,
    conditions=[
        Condition(
            'humidity level in lab',
            value=NominalReal(float(PPMS_ET_setup['humidity level in lab']),'kg/kg'),   
        ),
        Condition(
            'voltage orientation',
            value=NominalCategorical(PPMS_ET_setup['voltage orientation'])
        ),
    
    ],
    
    parameters=[
        Parameter(
            'current',
            value=NominalReal(float(PPMS_ET_setup['current']),'amp'),
        ),
        Parameter(
            'sample dimentions',
            value=NominalCategorical(PPMS_ET_setup['sample dimentions'])
        ),
        Parameter(
            'compound stochiometry',
            value=NominalCategorical(PPMS_ET_setup['compound stochiometry'])
        )
    ],
)

In [19]:
run_ET = make_instance(spec_ET)
run_ET.source = PerformedSource(PPMS_ET_setup['name'],PPMS_ET_setup['date'])

In [20]:
encoder = GEMDJson()

fn = 'ET_PPMS_process_template.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(temp_ET,indent=2))

fn = 'ET_PPMS_process_spec.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(spec_ET,indent=2))

fn = 'ET_PPMS_process_run.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(run_ET,indent=2))

entire_model_fn = 'ET_PPMS_entire_model.json'
with open(pathlib.Path()/entire_model_fn,'w') as fp :
    fp.write(encoder.dumps([run_ET],indent=2))


## Thermal Transport PPMS

In [24]:
PPMS_TT_setup = {
    'name': 'Julia',
    'date': '17/07/2022',
    #conditions
    'grease name': 'grease', #if no grease then put 'None'
    'grease amount': '8.0',# if no greae then put 'None'
    'humidity level in lab': '0.08',
    #parameters 
    'mass of sample': '8', #in g
    'molar mass': '60.8', #g/mol
    'current': '6',# in Amp
    'sample dimentions': '50:60', # number x: number y in mm
    'compound stochiometry': 'NO2' } #in capital letteres 

#should I add an extra category asking about whether they are looking for heat capacity or themral conductivity?

In [25]:
#set up templates
temp_TT = ProcessTemplate(
    'Thermal Transport PPMS setup',
    parameters=[
        
        ParameterTemplate('humidity level in lab', bounds=RealBounds(0.0,1.0,'kg/kg') ),
        
        ParameterTemplate('grease name', bounds=CategoricalBounds(['grease',]) ),
        
        ParameterTemplate('grease amount', bounds=RealBounds(0.0,10,'g') ),
    ],
)

In [26]:
#set up spec
spec_TT = ProcessSpec(
    name='Thermal Transport PPMS setup',
    template=temp_TT,
    conditions=[
        Condition(
            'humidity level in lab',
            value=NominalReal(float(PPMS_TT_setup['humidity level in lab']),'kg/kg'),   
        ),
        Condition(
            'grease name',
            value=NominalCategorical(PPMS_TT_setup['grease name'])
        ),
        Condition(
            'grease amount',
            value=NominalReal(float(PPMS_TT_setup['grease amount']),'g')
        ),
    ],
    
    parameters=[
         Parameter(
            'mass of sample',
            value=NominalReal(float(PPMS_TT_setup['mass of sample']),'g'),
        ),
         Parameter(
            'molar mass',
            value=NominalReal(float(PPMS_TT_setup['molar mass']),'g/mol'),
        ),
        Parameter(
            'current',
            value=NominalReal(float(PPMS_TT_setup['current']),'amp'),
        ),
        Parameter(
            'sample dimentions',
            value=NominalCategorical(PPMS_TT_setup['sample dimentions'])
        ),
        Parameter(
            'compound stochiometry',
            value=NominalCategorical(PPMS_TT_setup['compound stochiometry'])
        )
    ],
)

In [36]:
run_ET = make_instance(spec_TT)
run_ET.source = PerformedSource(PPMS_TT_setup['name'],PPMS_TT_setup['date'])

In [ ]:
encoder = GEMDJson()

fn = 'TT_PPMS_process_template.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(temp_TT,indent=2))

fn = 'TT_PPMS_process_spec.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(spec_TT,indent=2))

fn = 'TT_PPMS_process_run.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(run_TT,indent=2))

entire_model_fn = 'TT_PPMS_entire_model.json'
with open(pathlib.Path()/entire_model_fn,'w') as fp :
    fp.write(encoder.dumps([run_TT],indent=2))


## Magnetic PPMS and MPMS 

In [28]:
PPMS_MPMS_M_setup = {
    'name': 'Julia',
    'date': '17/07/2022',
    #conditions
    'plastic straw name': 'straw', 
    'plastic straw thickness': '0.2', #in mm
    'cling film name': 'cling that film',
    'cling film amount': '30:20',# number x: number y in mm
    'humidity level in lab': '0.08',
    'background magnitism': '25', #in micro Tesla
    #parameters 
    'mass of sample': '8', #in g
    'molar mass': '60.8', #g/mol
    'current': '6', #in Amp
    'type of current': 'AC', #either 'AC or 'DC'
    'sample dimentions': '50:60', # number x: number y in mm
    'compound stochiometry': 'NO2' } #in capital letteres 

In [29]:
#set up templates
temp_M = ProcessTemplate(
    'Magnetic PPMS and MPMS setup',
    parameters=[
        
        ParameterTemplate('humidity level in lab', bounds=RealBounds(0.0,1.0,'kg/kg') ),
        
        ParameterTemplate('plastic straw name', bounds=CategoricalBounds(['straw',]) ),
        
        ParameterTemplate('type of current', bounds=CategoricalBounds(['AC', 'DC']) ),
    ],
)

In [34]:
#set up spec
spec_M = ProcessSpec(
    name='Magnetic PPMS and MPMS setup',
    template=temp_M,
    conditions=[
        Condition(
            'humidity level in lab',
            value=NominalReal(float(PPMS_MPMS_M_setup['humidity level in lab']),'kg/kg'),   
        ),
        Condition(
            'plastic straw name',
            value=NominalCategorical(PPMS_MPMS_M_setup['plastic straw name'])
        ),
        Condition(
            'plastic straw thickness',
            value=NominalReal(float(PPMS_MPMS_M_setup['plastic straw thickness']),'g')
        ),
        Condition(
            'cling film name',
            value=NominalCategorical(PPMS_MPMS_M_setup['cling film name'])
        ),
        Condition(
            'cling film amount',
            value=NominalCategorical(PPMS_MPMS_M_setup['cling film amount'])
        ),
        Condition(
            'background magnitism',
            value=NominalReal(float(PPMS_MPMS_M_setup['background magnitism']), 'T') #does not take micro???
        ),
    ],
    
    parameters=[
         Parameter(
            'mass of sample',
            value=NominalReal(float(PPMS_MPMS_M_setup['mass of sample']),'g'),
        ),
         Parameter(
            'molar mass',
            value=NominalReal(float(PPMS_MPMS_M_setup['molar mass']),'g/mol'),
        ),
        Parameter(
            'current',
            value=NominalReal(float(PPMS_MPMS_M_setup['current']),'amp'),
        ),
        Parameter(
            'type of current',
            value=NominalCategorical(PPMS_MPMS_M_setup['type of current'])
        ),
        Parameter(
            'sample dimentions',
            value=NominalCategorical(PPMS_MPMS_M_setup['sample dimentions'])
        ),
        Parameter(
            'compound stochiometry',
            value=NominalCategorical(PPMS_MPMS_M_setup['compound stochiometry'])
        )
    ],
)

In [35]:
run_M = make_instance(spec_M)
run_M.source = PerformedSource(PPMS_MPMS_M_setup['name'],PPMS_MPMS_M_setup['date'])

In [ ]:
encoder = GEMDJson()

fn = 'PPMS_MPMS_M_process_template.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(temp_M,indent=2))

fn = 'PPMS_MPMS_M_process_spec.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(spec_M,indent=2))

fn = 'PPMS_MPMS_M_process_run.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(run_M,indent=2))

entire_model_fn = 'PPMS_MPMS_M_entire_model.json'
with open(pathlib.Path()/entire_model_fn,'w') as fp :
    fp.write(encoder.dumps([run_M],indent=2))
